In [1]:
import pandas as pd
import numpy as np

# ACCT NR

In [2]:
# GCIF Acct mapping - From EDW Bridge
gcifAcct = pd.read_csv(r"C:\Users\MR38804\Documents\EDW Transac\Cust List\GCIF ACCT 2212.csv", sep="|", dtype={'ACCT':'str'})

In [3]:
# NR Customer - From LoBs monitoring
nrCust = pd.read_excel(r'C:\Users\MR38804\Documents\EDW Transac\Cust List\2212 Dec NR Cust List.xlsx', sheet_name='Clean')

In [4]:
nrCust[nrCust['GCIF_NO'].duplicated()]

,GCIF_NO,CUST_NAME,LoB,REGION


In [ ]:
nrCust

In [6]:
# Merge gcif acct mapping to NR GCIF list
acctNR = pd.merge(nrCust, gcifAcct, left_on=['GCIF_NO'], right_on=['GCIF_NO'], how='left')

In [7]:
acctNR['ACCT'] = acctNR['ACCT'].astype('str').str.zfill(24)

In [8]:
acctNR = acctNR[['ACCT', 'GCIF_NO', 'CUST_NAME', 'LoB', 'REGION']]

In [9]:
acctNR = acctNR[~acctNR['ACCT'].duplicated()]

In [ ]:
acctNR

In [11]:
acctNR[acctNR['CUST_NAME']=='RDT BNP PARI']

,ACCT,GCIF_NO,CUST_NAME,LoB,REGION
5215,000000000000000000000nan,G209619386,RDT BNP PARI,GB,NaN


In [12]:
acctNR = acctNR[~(acctNR['ACCT']=='00000000000000000nan')]

In [ ]:
acctNR

# BKCD

In [14]:
# Read Bank Code and SWIFT code translate
BKCD = pd.read_excel(r'C:\Users\MR38804\Documents\EDW Transac\Bank Code\BKCD 2212.xlsx')

In [15]:
BKCD

,Kode Bank Indonesia Fixed,Kode Bank Indonesia,BIC(Self/EDW),BIC(google),NAMA BANK ORI,TABEL SANDI,NAMA FINAL
0,0,NaN,NaN,AGTBIDJA,BANK BRI AGRO,AGRONIAGA,AGRONIAGA
1,0,NaN,NaN,ALTOIDJ1,ALTO NETWORK,NaN,ALTO NETWORK
2,30,30.0,NaN,AEIBPKK1LHE,AMERICAN EXPRESS BANK LTD,NaN,AMERICAN EXPRESS BANK
3,0,NaN,NaN,ATJSIDJ2,NaN,NaN,ARTAJASA PEMBAYARAN ELEKTRONIK
4,0,NaN,NaN,BKKBIDJA,BANGKOK BANK,BANGKOK BANK,BANGKOK BANK
...,...,...,...,...,...,...,...
189,0,NaN,NaN,FASTIDJA,NaN,NaN,BANK INDONESIA FAST PAYMENT
190,0,NaN,NaN,SKNAIDJ1,NaN,NaN,SKNBI
191,0,NaN,NaN,INDOIDJP,NaN,NaN,BANK INDONESIA - PENYELENGGARA RTGS
192,422,422.0,NaN,NaN,NaN,NaN,BANK BRI SYARIAH


In [16]:
BKCD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Kode Bank Indonesia Fixed  194 non-null    int64  
 1   Kode Bank Indonesia        163 non-null    float64
 2   BIC(Self/EDW)              79 non-null     object 
 3   BIC(google)                182 non-null    object 
 4   NAMA BANK ORI              186 non-null    object 
 5   TABEL SANDI                153 non-null    object 
 6   NAMA FINAL                 194 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 10.7+ KB


In [17]:
# Create unique BKCD table for merging
BKCDBIC = BKCD[['BIC(google)','NAMA FINAL']].drop_duplicates(subset='BIC(google)', keep='first')

In [18]:
BKCDBIC.dropna(inplace=True)

In [19]:
BKCDBIC.reset_index(drop=True, inplace=True)

In [20]:
BKCDBIC.columns = ['BIC', 'BANK NAME']

In [21]:
BKCDBIC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   BIC        158 non-null    object
 1   BANK NAME  158 non-null    object
dtypes: object(2)
memory usage: 2.6+ KB


In [22]:
BKCDBIC

,BIC,BANK NAME
0,AGTBIDJA,AGRONIAGA
1,ALTOIDJ1,ALTO NETWORK
2,AEIBPKK1LHE,AMERICAN EXPRESS BANK
3,ATJSIDJ2,ARTAJASA PEMBAYARAN ELEKTRONIK
4,BKKBIDJA,BANGKOK BANK
...,...,...
153,SCBLIDJX,STANDARD CHARTERED BANK
154,BBIJIDJA,UOB INDONESIA
155,FASTIDJA,BANK INDONESIA FAST PAYMENT
156,SKNAIDJ1,SKNBI


In [23]:
BKCD = BKCD[['Kode Bank Indonesia Fixed','NAMA FINAL']].drop_duplicates(subset='Kode Bank Indonesia Fixed', keep='first')

In [24]:
BKCD.columns = ['BANK CODE', 'BANK NAME']

In [25]:
BKCD['BANK CODE'] = BKCD['BANK CODE'].astype('str').str.zfill(3)

In [26]:
BKCD = BKCD[BKCD['BANK CODE']!='000']

In [27]:
BKCD

,BANK CODE,BANK NAME
2,030,AMERICAN EXPRESS BANK
5,040,BANGKOK BANK
6,052,BANK ABN AMRO
7,116,BANK ACEH
8,525,BANK AKITA
...,...,...
183,042,MUFG BANK
185,520,PRIMA MASTER
187,050,STANDARD CHARTERED BANK
188,023,UOB INDONESIA


# ORTGS

In [28]:
# read ortgs raw
ortgs = pd.read_csv(r"C:\Users\MR38804\Documents\EDW Transac\ORTGS\ORTGS_2212.txt", sep='\t')

In [29]:
ortgs

,BASE_DT,FR_MEMBER,Unnamed: 2,TO_MEMBER,Unnamed: 4,AMOUNT,FR_ACC_NO,TO_ACC_NM,ORIGIN_NM,GCIF_NO,ULT_BEN_ACC,ULT_BEN_NM
0,20221207,IBBKIDJA,BANK BII MAYBANK,MEGAIDJA,BANK MEGA,5.500000e+09,8.132002e+09,"PT. BANK MEGA, TBK",000000000000008132001952 - ARDY WARDHIKA ...,G600512818,1.252E+13,ARDY WARDHIKA or JOVAN CHANDRA ARDY
1,20221214,IBBKIDJA,BANK BII MAYBANK,SYCAIDJ1,BANK BCA SYARIAH,5.000000e+09,8.132002e+09,PT. BANK BCA SYARIAH,000000000000008132001952 - ARDY WARDHIKA ...,G600512818,30003017,ARDY WARDHIKA
2,20221214,IBBKIDJA,BANK BII MAYBANK,MUABIDJA,BANK MUAMALAT,5.000000e+08,8.132002e+09,"PT. BANK MUAMALAT INDONESIA, TBK",000000000000008132001952 - ARDY WARDHIKA ...,G600512818,3300234567,PT. MANDIRI SEKURITAS
3,20221219,IBBKIDJA,BANK BII MAYBANK,MEGAIDJA,BANK MEGA,2.500000e+09,8.132002e+09,"PT. BANK MEGA, TBK",000000000000008132001952 - ARDY WARDHIKA ...,G600512818,1.252E+13,ARDY WARDHIKA or JOVAN CHANDRA ARDY
4,20221216,IBBKIDJA,BANK BII MAYBANK,CENAIDJA,BANK BCA,2.000000e+08,1.774005e+09,"PT. BANK CENTRAL ASIA, TBK",000000000000001774005275 - SURYA HARTO ...,G804704039,6090325939,SURYA HARTO DR IR
...,...,...,...,...,...,...,...,...,...,...,...,...
9912,20221230,IBBKIDJA,BANK BII MAYBANK,CENAIDJA,BANK BCA,1.110581e+08,2.999990e+09,"PT. BANK CENTRAL ASIA, TBK",000000000000002999990030 - PENGANGKUTAN KEKAL ...,NaN,8605999464,PT.MAJA SAMUDERA KIRANA
9913,20221230,IBBKIDJA,BANK BII MAYBANK,BBIJIDJA,BANK�UOB INDONESIA,1.050129e+08,2.999990e+09,PT. BANK UOB INDONESIA,000000000000002999990030 - MOVING WALLS SDN. B...,NaN,4653004650,MOVING WALLS INDONESIA PT
9914,20221230,IBBKIDJA,BANK BII MAYBANK,BNINIDJA,BANK BNI,1.075864e+08,2.999990e+09,PT. BANK NEGARA INDONESIA (PERSERO),000000000000002999990030 - PERUSAHAAN MEKASSAR...,NaN,366554486,PT BINA KRIDATAMA LESTARI
9915,20221202,IBBKIDJA,BANK BII MAYBANK,CENAIDJA,BANK BCA,1.000000e+09,2.999990e+08,"PT. BANK CENTRAL ASIA, TBK",000000000000000299999048 - MAYBANK SALATIGA ...,NaN,523014000,BCA CABANG PEMUDA SEMARANG


In [30]:
ortgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9917 entries, 0 to 9916
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BASE_DT      9917 non-null   int64  
 1   FR_MEMBER    9917 non-null   object 
 2   Unnamed: 2   9917 non-null   object 
 3   TO_MEMBER    9917 non-null   object 
 4   Unnamed: 4   9917 non-null   object 
 5   AMOUNT       9917 non-null   float64
 6   FR_ACC_NO    9917 non-null   float64
 7   TO_ACC_NM    9917 non-null   object 
 8   ORIGIN_NM    9917 non-null   object 
 9   GCIF_NO      7955 non-null   object 
 10  ULT_BEN_ACC  9286 non-null   object 
 11  ULT_BEN_NM   9309 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 929.8+ KB


In [ ]:
# show duplicated rows
ortgs[ortgs.duplicated(keep=False)]

In [32]:
# drop duplicates and null GCIF
ortgs = ortgs[~ortgs['GCIF_NO'].isnull()]
ortgs.drop_duplicates(inplace=True)

<ipython-input-32-ee2a4c7c9f8f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ortgs.drop_duplicates(inplace=True)


In [33]:
# modify/comment out as necessary
ortgs.drop(columns=['Unnamed: 2', 'Unnamed: 4'], inplace=True)

c:\Users\MR38804\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
ortgs

In [35]:
ortgs.columns

Index(['BASE_DT', 'FR_MEMBER', 'TO_MEMBER', 'AMOUNT', 'FR_ACC_NO', 'TO_ACC_NM',
       'ORIGIN_NM', 'GCIF_NO', 'ULT_BEN_ACC', 'ULT_BEN_NM'],
      dtype='object')

In [36]:
# add source bank
ortgs['FR_BK'] = 'BANK BII MAYBANK'

<ipython-input-36-966c475eefdc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ortgs['FR_BK'] = 'BANK BII MAYBANK'


In [37]:
# merge to obtain bank name
ortgsBK = pd.merge(ortgs, BKCDBIC, left_on=['TO_MEMBER'], right_on=['BIC'], how='left')

In [ ]:
ortgsBK

In [39]:
# merge to obtain NR only
ortgsFinal = pd.merge(ortgsBK, nrCust, left_on = ['GCIF_NO'], right_on=['GCIF_NO'], how='left')

In [ ]:
ortgsFinal

In [ ]:
ortgsFinal = ortgsFinal[ortgsFinal['CUST_NAME'].notna()]
ortgsFinal

In [42]:
ortgsFinal.columns

Index(['BASE_DT', 'FR_MEMBER', 'TO_MEMBER', 'AMOUNT', 'FR_ACC_NO', 'TO_ACC_NM',
       'ORIGIN_NM', 'GCIF_NO', 'ULT_BEN_ACC', 'ULT_BEN_NM', 'FR_BK', 'BIC',
       'BANK NAME', 'CUST_NAME', 'LoB', 'REGION'],
      dtype='object')

In [43]:
# rename columns
ortgsFinal.columns = ['BASE DATE', 'FR_MEMBER', 'TO_MEMBER', 'AMT', 'FR_ACC_NO', 'TO_ACC_NM',
       'ORIGIN_NM', 'GCIF', 'ULT_BEN_ACC', 'SENDER/RECIPIENT NAME', 'FR_BK', 'BIC',
       'SENDER/RECIPIENT BANK', 'CUST NAME', 'LOB', 'REGION']

In [44]:
# add transaction type and channel
ortgsFinal['TYPE'] = 'OUTGOING'
ortgsFinal['CHANNEL'] = 'RTGS'

<ipython-input-44-4db2db1325b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ortgsFinal['TYPE'] = 'OUTGOING'
<ipython-input-44-4db2db1325b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ortgsFinal['CHANNEL'] = 'RTGS'


In [45]:
# save only relevant columns
ortgsFinal = ortgsFinal[['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME', 
'SENDER/RECIPIENT BANK', 'REGION', 'AMT', 'TYPE', 'CHANNEL']]

In [ ]:
ortgsFinal

# IRTGS

In [47]:
# read irtgs raw
irtgs = pd.read_csv(r"C:\Users\MR38804\Documents\EDW Transac\IRTGS\IRTGS_2212.txt", sep='\t', dtype={'ULT_BEN_ACC':'str'})
irtgs['ULT_BEN_ACC'] = irtgs['ULT_BEN_ACC'].str.zfill(24)

In [48]:
irtgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12196 entries, 0 to 12195
Data columns (total 38 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BASE_DT      12196 non-null  int64  
 1   REC_TYPE     12196 non-null  int64  
 2   TRAN_CODE    12196 non-null  int64  
 3   BOR          12196 non-null  int64  
 4   URN_INFO     12196 non-null  int64  
 5   ERR_FLAG     12196 non-null  int64  
 6   OSN          12196 non-null  int64  
 7   FR_MEMBER    12196 non-null  object 
 8   TO_MEMBER    12196 non-null  object 
 9   REC_BK_BR    0 non-null      float64
 10  TRN          12196 non-null  object 
 11  REL_TRN      12196 non-null  object 
 12  AMOUNT       12196 non-null  float64
 13  VAL_DATE     12196 non-null  int64  
 14  SENDER_REF   0 non-null      float64
 15  REC_REF      0 non-null      float64
 16  DEAL_CODE    0 non-null      float64
 17  FR_ACC_NO    12187 non-null  float64
 18  FR_ACC_NM    12196 non-null  object 
 19  TO_A

In [49]:
# extract origin acc
irtgs['ORIGIN_ACC'] = irtgs['ORIGIN_NM'].str.extract('(\d*)\s+-')

In [50]:
# extract origin name
irtgs['ORIGIN_NAME'] = irtgs['ORIGIN_NM'].str.extract('-\s(\D*\S*)')

In [51]:
# cleanse origin name
irtgs['ORIGIN_NAME'] = irtgs['ORIGIN_NAME'].str[:35]
irtgs['ORIGIN_NAME'] = irtgs['ORIGIN_NAME'].str.rstrip()

In [52]:
irtgs['FR_MEMBER'] = irtgs['FR_MEMBER'].str.rstrip()

In [53]:
irtgs[irtgs.duplicated()]

,BASE_DT,REC_TYPE,TRAN_CODE,BOR,URN_INFO,ERR_FLAG,OSN,FR_MEMBER,TO_MEMBER,REC_BK_BR,...,INT_RT,INT_RT_DEC,PERIOD,KODE_CABANG,NAS,REF,CONFIRM,ORG_PRC_DT,ORIGIN_ACC,ORIGIN_NAME


In [54]:
# merge to obtain bank name
irtgsBK = pd.merge(irtgs, BKCDBIC, left_on=['FR_MEMBER'], right_on=['BIC'], how='left')

In [ ]:
irtgsBK

In [56]:
# check if there is any NA in bank name or not
irtgsBK.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12196 entries, 0 to 12195
Data columns (total 42 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BASE_DT      12196 non-null  int64  
 1   REC_TYPE     12196 non-null  int64  
 2   TRAN_CODE    12196 non-null  int64  
 3   BOR          12196 non-null  int64  
 4   URN_INFO     12196 non-null  int64  
 5   ERR_FLAG     12196 non-null  int64  
 6   OSN          12196 non-null  int64  
 7   FR_MEMBER    12196 non-null  object 
 8   TO_MEMBER    12196 non-null  object 
 9   REC_BK_BR    0 non-null      float64
 10  TRN          12196 non-null  object 
 11  REL_TRN      12196 non-null  object 
 12  AMOUNT       12196 non-null  float64
 13  VAL_DATE     12196 non-null  int64  
 14  SENDER_REF   0 non-null      float64
 15  REC_REF      0 non-null      float64
 16  DEAL_CODE    0 non-null      float64
 17  FR_ACC_NO    12187 non-null  float64
 18  FR_ACC_NM    12196 non-null  object 
 19  TO_A

In [57]:
# join acct no to obtain cust info
irtgsFinal = pd.merge(irtgsBK, acctNR, left_on = ['ULT_BEN_ACC'], right_on=['ACCT'], how='left')

In [58]:
irtgsFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12196 entries, 0 to 12195
Data columns (total 47 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BASE_DT      12196 non-null  int64  
 1   REC_TYPE     12196 non-null  int64  
 2   TRAN_CODE    12196 non-null  int64  
 3   BOR          12196 non-null  int64  
 4   URN_INFO     12196 non-null  int64  
 5   ERR_FLAG     12196 non-null  int64  
 6   OSN          12196 non-null  int64  
 7   FR_MEMBER    12196 non-null  object 
 8   TO_MEMBER    12196 non-null  object 
 9   REC_BK_BR    0 non-null      float64
 10  TRN          12196 non-null  object 
 11  REL_TRN      12196 non-null  object 
 12  AMOUNT       12196 non-null  float64
 13  VAL_DATE     12196 non-null  int64  
 14  SENDER_REF   0 non-null      float64
 15  REC_REF      0 non-null      float64
 16  DEAL_CODE    0 non-null      float64
 17  FR_ACC_NO    12187 non-null  float64
 18  FR_ACC_NM    12196 non-null  object 
 19  TO_A

In [59]:
# nr only
irtgsFinal= irtgsFinal[irtgsFinal['GCIF_NO'].notnull()]

In [60]:
irtgsFinal.columns

Index(['BASE_DT', 'REC_TYPE', 'TRAN_CODE', 'BOR', 'URN_INFO', 'ERR_FLAG',
       'OSN', 'FR_MEMBER', 'TO_MEMBER', 'REC_BK_BR', 'TRN', 'REL_TRN',
       'AMOUNT', 'VAL_DATE', 'SENDER_REF', 'REC_REF', 'DEAL_CODE', 'FR_ACC_NO',
       'FR_ACC_NM', 'TO_ACC_NO', 'TO_ACC_NM', 'PAY_DET', 'MEMBER_INFO',
       'ORIGIN_NM', 'ULT_BEN_ACC', 'ULT_BEN_NM', 'ULT_BEN_BL', 'CUR', 'EXC_RT',
       'EXC_RT_DEC', 'INT_RT', 'INT_RT_DEC', 'PERIOD', 'KODE_CABANG', 'NAS',
       'REF', 'CONFIRM', 'ORG_PRC_DT', 'ORIGIN_ACC', 'ORIGIN_NAME', 'BIC',
       'BANK NAME', 'ACCT', 'GCIF_NO', 'CUST_NAME', 'LoB', 'REGION'],
      dtype='object')

In [61]:
# only take columns as needed
irtgsFinal = irtgsFinal[['BASE_DT', 'GCIF_NO', 'CUST_NAME', 'LoB', 'ORIGIN_NAME', 'BANK NAME', 'REGION', 'AMOUNT']]

In [62]:
# add type and channel of transactions
irtgsFinal['TYPE'] = 'INCOMING'
irtgsFinal['CHANNEL'] = 'RTGS'

In [63]:
# finalize columns
irtgsFinal.columns = ['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME', 
'SENDER/RECIPIENT BANK', 'REGION', 'AMT', 'TYPE', 'CHANNEL']

In [ ]:
irtgsFinal

# OSKN

In [65]:
# read skn data from source table
osknSource1 = pd.read_excel(r"C:\Users\MR38804\Documents\EDW Transac\OSKN\Dec'22\2022 SUM_STSRTEFTD.xls", sheet_name="Sheet 1", usecols=['BASE_DT', 'REC_TP', 'REFF_NUM_TP', 'ACCT_SEND_NO'], dtype={'ACCT_SEND_NO':'object'})
osknSource2 = pd.read_excel(r"C:\Users\MR38804\Documents\EDW Transac\OSKN\Dec'22\2022 SUM_STSRTEFTD.xls", sheet_name="Sheet 2", usecols=['BASE_DT', 'REC_TP', 'REFF_NUM_TP', 'ACCT_SEND_NO'], dtype={'ACCT_SEND_NO':'object'})

In [66]:
# append data from 2 sheets
osknSource = osknSource1.append(osknSource2)

In [67]:
osknSource.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122629 entries, 0 to 57629
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   BASE_DT       122629 non-null  int64 
 1   REC_TP        122629 non-null  object
 2   REFF_NUM_TP   122629 non-null  object
 3   ACCT_SEND_NO  108748 non-null  object
dtypes: int64(1), object(3)
memory usage: 4.7+ MB


In [68]:
# filter for skn only (exclude RTGS)
osknSource = osknSource[osknSource['REC_TP']=='L ']

In [69]:
# check for duplicates
osknSource[osknSource.duplicated(keep=False)]

,BASE_DT,REC_TP,REFF_NUM_TP,ACCT_SEND_NO


In [ ]:
osknSource

In [71]:
# change acct send no type to string
osknSource['ACCT_SEND_NO'] = osknSource['ACCT_SEND_NO'].astype(str)
# osknSource['ACCT_SEND_NO'] = osknSource['ACCT_SEND_NO'].str[-14:]

In [72]:
# read skn from edwb
osknEDWB = pd.read_csv(r"C:\Users\MR38804\Documents\EDW Transac\OSKN\Dec'22\TRXEFACMM 2212.csv", sep='|')

In [73]:
osknEDWB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122629 entries, 0 to 122628
Data columns (total 57 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   BASE_DT             122629 non-null  int64  
 1   REC_TP              122629 non-null  object 
 2   CURR_CD             122629 non-null  int64  
 3   REFF_NUM_TP         122629 non-null  object 
 4   BR_SEND             122629 non-null  int64  
 5   CURR_SEND           122629 non-null  int64  
 6   DT_SEND             122629 non-null  int64  
 7   TM_SEND             122629 non-null  int64  
 8   TERM_SEND           122629 non-null  object 
 9   BR_RECEIVE          122629 non-null  int64  
 10  CURR_RECEIVE        122629 non-null  int64  
 11  DT_RECEIVE          122629 non-null  int64  
 12  TM_RECEIVE          122629 non-null  int64  
 13  TERM_RECEIVE        122629 non-null  object 
 14  SRC_CD              122629 non-null  object 
 15  STATUS              122629 non-nul

In [74]:
# filter skn only (exclude rtgs)
osknEDWB = osknEDWB[osknEDWB['REC_TP']=='L ']

In [75]:
osknEDWB['REFF_NUM_TP'] = osknEDWB['REFF_NUM_TP'].astype(str)

In [76]:
# merge SKN Source and EDWB
osknComb = pd.merge(osknEDWB, osknSource, left_on=['BASE_DT','REFF_NUM_TP'], right_on=['BASE_DT','REFF_NUM_TP'], how='left')

In [77]:
# check acct_send_no on join result
osknComb['ACCT_SEND_NO'].count()

99242

In [78]:
# check acct_send_no on source
osknSource['ACCT_SEND_NO'].count()

99242

In [79]:
# take necessary columns only
osknComb = osknComb[['BASE_DT','REFF_NUM_TP', 'AMT','NM_SENDER_1','ACCT_RECEIVER_ZERO','NM_RECEIVER_1',
'BIC_CD','ACCT_SEND_NO']]

In [80]:
osknComb.columns = ['BASE_DT','REFF_NO', 'AMT','NM_SENDER','ACCT_RECEIVER','NM_RECEIVER',
'BIC_CD','ACCT_SEND_NO']

In [ ]:
osknComb

In [82]:
oskoutm = pd.read_csv(r"C:\Users\MR38804\Documents\EDW Transac\OSKN\Dec'22\TRXSTSKOUTM 2212.csv", sep='|', dtype={'OP_AC_NO':'str'})

c:\Users\MR38804\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
oskoutm

In [84]:
oskoutm['REF_NO'] = oskoutm['REF_NO'].astype(str)
oskoutm['OP_AC_NO'] = oskoutm['OP_AC_NO'].astype(str)

In [85]:
# oskoutm['NO_LN'] = oskoutm['ACCT_NO'].str.len()

In [86]:
# oskoutm.sort_values(by='NO_LN', ascending=False)

In [87]:
# take necessary columns only
oskoutm = oskoutm[['BASE_DT','REF_NO', 'OP_NM','CU_NM','ACCT_NO','AMT',
'OP_AC_NO','BIC']]

In [88]:
# rename columns
oskoutm.columns = ['BASE_DT','REFF_NO', 'NM_SENDER','NM_RECEIVER','ACCT_RECEIVER','AMT',
'ACCT_SEND_NO','BIC_CD']

In [89]:
oskoutm['ACCT_SEND_NO'] = oskoutm['ACCT_SEND_NO'].astype('str').str.zfill(24)

<ipython-input-89-ec3750dd9bdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oskoutm['ACCT_SEND_NO'] = oskoutm['ACCT_SEND_NO'].astype('str').str.zfill(24)


In [ ]:
oskoutm

In [91]:
oskoutm = oskoutm[~(oskoutm[['REFF_NO', 'NM_SENDER', 'NM_RECEIVER', 'ACCT_RECEIVER', 'AMT', 'ACCT_SEND_NO', 'BIC_CD']].duplicated(keep='last'))]

In [92]:
# append all skn
osknAll = osknComb.append(oskoutm, ignore_index=True)

In [93]:
osknAll[osknAll.duplicated(keep=False)]

,BASE_DT,REFF_NO,AMT,NM_SENDER,ACCT_RECEIVER,NM_RECEIVER,BIC_CD,ACCT_SEND_NO


In [94]:
# osknAll.drop_duplicates(inplace=True)

In [95]:
osknAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115360 entries, 0 to 115359
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   BASE_DT        115360 non-null  int64  
 1   REFF_NO        115360 non-null  object 
 2   AMT            115360 non-null  float64
 3   NM_SENDER      115360 non-null  object 
 4   ACCT_RECEIVER  115360 non-null  object 
 5   NM_RECEIVER    115360 non-null  object 
 6   BIC_CD         115360 non-null  object 
 7   ACCT_SEND_NO   115360 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 7.0+ MB


In [96]:
osknBK = pd.merge(osknAll, BKCDBIC, left_on=['BIC_CD'], right_on=['BIC'], how='left')

In [97]:
osknBK['BANK NAME'].count()

115360

In [ ]:
osknBK

In [99]:
osknBK['ACCT_SEND_NO'] = osknBK['ACCT_SEND_NO'].str.zfill(24)

In [100]:
osknFinal = pd.merge(osknBK, acctNR, left_on = ['ACCT_SEND_NO'], right_on=['ACCT'], how='left')

In [ ]:
osknFinal

In [102]:
osknFinal.columns

Index(['BASE_DT', 'REFF_NO', 'AMT', 'NM_SENDER', 'ACCT_RECEIVER',
       'NM_RECEIVER', 'BIC_CD', 'ACCT_SEND_NO', 'BIC', 'BANK NAME', 'ACCT',
       'GCIF_NO', 'CUST_NAME', 'LoB', 'REGION'],
      dtype='object')

In [103]:
osknFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115360 entries, 0 to 115359
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   BASE_DT        115360 non-null  int64  
 1   REFF_NO        115360 non-null  object 
 2   AMT            115360 non-null  float64
 3   NM_SENDER      115360 non-null  object 
 4   ACCT_RECEIVER  115360 non-null  object 
 5   NM_RECEIVER    115360 non-null  object 
 6   BIC_CD         115360 non-null  object 
 7   ACCT_SEND_NO   115360 non-null  object 
 8   BIC            115360 non-null  object 
 9   BANK NAME      115360 non-null  object 
 10  ACCT           91662 non-null   object 
 11  GCIF_NO        91662 non-null   object 
 12  CUST_NAME      91662 non-null   object 
 13  LoB            91662 non-null   object 
 14  REGION         75832 non-null   object 
dtypes: float64(1), int64(1), object(13)
memory usage: 14.1+ MB


In [104]:
# osknFinal = osknFinal[~osknFinal[['REFF_NO', 'AMT', 'NM_SENDER', 'ACCT_RECEIVER',
#       'NM_RECEIVER', 'BIC_CD', 'ACCT_SEND_NO', 'BIC', 'BANK NAME', 'ACCT',
#       'GCIF_NO', 'CUST_NAME', 'LoB', 'REGION']].duplicated(keep='last')]

In [105]:
osknFinal['CUST_NAME'].value_counts()

WAHANA OTTOMITRA MULTIARTHA TBK, PT    4398
MAYBANK INDONESIA FINANCE, PT          4101
REKENING PENAMPUNGAN SECURITIE         3798
VICTORY RETAILINDO, PT                 2312
HARJA GUNATAMA LESTARI, PT             1097
                                       ... 
BIMA SAKTI LOGISTIK                       1
VIVA KONET INTERNATIONAL, PT              1
SONGPENG VALASINDO, PT                    1
FERRO SERVITA, PT                         1
ZENITH MATERIAL SOLUTION, PT              1
Name: CUST_NAME, Length: 3488, dtype: int64

In [106]:
osknFinal = osknFinal[osknFinal['CUST_NAME'].notna()]

In [107]:
osknFinal.columns

Index(['BASE_DT', 'REFF_NO', 'AMT', 'NM_SENDER', 'ACCT_RECEIVER',
       'NM_RECEIVER', 'BIC_CD', 'ACCT_SEND_NO', 'BIC', 'BANK NAME', 'ACCT',
       'GCIF_NO', 'CUST_NAME', 'LoB', 'REGION'],
      dtype='object')

In [108]:
osknFinal = osknFinal[['BASE_DT', 'GCIF_NO', 'CUST_NAME', 'LoB', 'NM_RECEIVER', 'BANK NAME',
'REGION', 'AMT']]

In [109]:
osknFinal.columns = ['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME', 
'SENDER/RECIPIENT BANK', 'REGION', 'AMT']

In [110]:
osknFinal['TYPE'] = 'OUTGOING'
osknFinal['CHANNEL'] = 'SKN'

In [ ]:
osknFinal

# ISKN

In [112]:
iskn = pd.read_csv(r'C:\Users\MR38804\Documents\EDW Transac\ISKN\TRXSTSKINM_2212.csv', sep='|', dtype={'SENDER_CLEARING_CD':'str', 'BENFRY_ACCT':'str'})

In [113]:
iskn

,REF_NO,SENDER_NM,BENFRY_NM,BENFRY_ACCT,AMT,SENDER_CLEARING_CD
0,000002475728,MCNICHOLL CONSULTING LLC,ANDREAS KOSASIH,1406022408,9.971000e+05,0330301
1,000002554616,PT. LODGING PARTNER SERVICES INDONESIA,PT METRODATA ELECTRONICS TBK,2185526801,2.406250e+07,0330301
2,000002562751,PAYPAL PTE LTD,I GST KETUT SUASTAMA,1768513028,9.320712e+05,0330301
3,000002594287,PAYPAL PTE LTD,PT AYUNAN BULAN INTERNATIONAL,2177030555,2.184462e+07,0330301
4,000002641816,PT. HERBALIFE INDONESIA,AI NIKMAH SITI FATIMAH,1402055332,1.828508e+06,0330301
...,...,...,...,...,...,...
439988,PPU.;033J-0057,PT WAHANA OTTOMITRA MULTIARTHA,PT WAHANA OTTOMITRA MULTIARTHA,2003565408,3.500000e+05,0140397
439989,PPU.=VFY5-0001,ERA MANDIRI CEMERLANG PT,A ROSITA,1518073945,5.000000e+08,0140397
439990,PPU.=VKFL-0011,PANGAN AGRO LESTARI PT,PT PANGAN AGRO LESTARI,2052128899,1.000000e+09,0140397
439991,PPU.=VWBF-0217,ELUON PT,PT. Indo Buana Berkah,2742000506,7.718869e+07,0140397


In [114]:
iskn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439993 entries, 0 to 439992
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   REF_NO              439993 non-null  object 
 1   SENDER_NM           439984 non-null  object 
 2   BENFRY_NM           439993 non-null  object 
 3   BENFRY_ACCT         439981 non-null  object 
 4   AMT                 439993 non-null  float64
 5   SENDER_CLEARING_CD  439993 non-null  object 
dtypes: float64(1), object(5)
memory usage: 20.1+ MB


In [115]:
iskn[['BK_CD']] = iskn['SENDER_CLEARING_CD'].str[:3]

In [116]:
iskn['BENFRY_ACCT'] = iskn['BENFRY_ACCT'].str.zfill(24)

In [117]:
iskn[iskn.duplicated()]

,REF_NO,SENDER_NM,BENFRY_NM,BENFRY_ACCT,AMT,SENDER_CLEARING_CD,BK_CD


In [ ]:
iskn

In [119]:
isknBK = pd.merge(iskn, BKCD, left_on='BK_CD', right_on='BANK CODE', how='left')

In [120]:
isknBK.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439993 entries, 0 to 439992
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   REF_NO              439993 non-null  object 
 1   SENDER_NM           439984 non-null  object 
 2   BENFRY_NM           439993 non-null  object 
 3   BENFRY_ACCT         439981 non-null  object 
 4   AMT                 439993 non-null  float64
 5   SENDER_CLEARING_CD  439993 non-null  object 
 6   BK_CD               439993 non-null  object 
 7   BANK CODE           439993 non-null  object 
 8   BANK NAME           439993 non-null  object 
dtypes: float64(1), object(8)
memory usage: 33.6+ MB


In [121]:
isknFinal = pd.merge(isknBK, acctNR, left_on=['BENFRY_ACCT'], right_on=['ACCT'], how='left')

In [ ]:
isknFinal

In [123]:
isknFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439993 entries, 0 to 439992
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   REF_NO              439993 non-null  object 
 1   SENDER_NM           439984 non-null  object 
 2   BENFRY_NM           439993 non-null  object 
 3   BENFRY_ACCT         439981 non-null  object 
 4   AMT                 439993 non-null  float64
 5   SENDER_CLEARING_CD  439993 non-null  object 
 6   BK_CD               439993 non-null  object 
 7   BANK CODE           439993 non-null  object 
 8   BANK NAME           439993 non-null  object 
 9   ACCT                218714 non-null  object 
 10  GCIF_NO             218714 non-null  object 
 11  CUST_NAME           218714 non-null  object 
 12  LoB                 218714 non-null  object 
 13  REGION              204507 non-null  object 
dtypes: float64(1), object(13)
memory usage: 50.4+ MB


In [124]:
isknFinal = isknFinal[isknFinal['CUST_NAME'].notna()]

In [125]:
isknFinal['BASE DATE'] = '20221200'

<ipython-input-125-d8c94598f5f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isknFinal['BASE DATE'] = '20221200'


In [126]:
isknFinal = isknFinal[['BASE DATE', 'GCIF_NO', 'CUST_NAME', 'LoB', 'SENDER_NM', 'BANK NAME',
'REGION', 'AMT']]

In [127]:
isknFinal.columns = ['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME', 
'SENDER/RECIPIENT BANK', 'REGION', 'AMT']

In [128]:
isknFinal['TYPE'] = 'INCOMING'
isknFinal['CHANNEL'] = 'SKN'

In [ ]:
isknFinal

# REMITTANCE

In [130]:
tt = pd.read_excel(r'C:\Users\MR38804\Documents\EDW Transac\Remittance\In Out Remittance as per Dec 22 Ori.xlsx')

In [131]:
tt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13094 entries, 0 to 13093
Data columns (total 82 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Periode             13094 non-null  int64  
 1   GCIF                12877 non-null  object 
 2   TRX_SEQ             6024 non-null   float64
 3   MTKODE              13094 non-null  object 
 4   RECBIC              13094 non-null  object 
 5   SENBIC              13094 non-null  object 
 6   SW20                13094 non-null  object 
 7   KD CAB ORI          13094 non-null  int64  
 8   new branch code     13094 non-null  int64  
 9   REG NAME            13094 non-null  object 
 10  CIF                 12877 non-null  float64
 11  lob                 13094 non-null  object 
 12  LOB Name            13094 non-null  object 
 13  Remark              13094 non-null  object 
 14  SWBHSN              13094 non-null  int64  
 15  SW108               7070 non-null   object 
 16  SW32

In [132]:
tt.columns

Index(['Periode', 'GCIF', 'TRX_SEQ', 'MTKODE', 'RECBIC', 'SENBIC', 'SW20',
       'KD CAB ORI', 'new branch code', 'REG NAME', 'CIF', 'lob', 'LOB Name',
       'Remark', 'SWBHSN', 'SW108', 'SW32D', 'SW32CURR', 'kurs', 'SW32AMT',
       'SW32AMT in IDR', 'SW32AMT in USD', 'tiering', 'SW50_1',
       'SW50_1 (ac repair)', 'LENGTH', 'For CA Facility', 'SW50_2', 'SW50_3',
       'SW50_4', 'SW52A', 'SW52_2', 'SW52_3', 'SW52_4', 'SW53A', 'SW53_2',
       'SW53_3', 'SW53_4', 'SW54A', 'SW54_2', 'SW54_3', 'SW54_4', 'SW56A',
       'SW56_2', 'SW56_3', 'SW56_4', 'SW57_1', 'country code', 'country name',
       'SW57_2', 'SW57_3', 'SW57_4', 'SW59_1', 'SW59_1(acrepair)', 'Length',
       'SW59_2', 'SW59_3', 'SW59_4', 'SW70_1', 'SW70_2', 'SW70_3', 'SW70_4',
       'SW71A', 'SW72_1', 'SW72_2', 'SW72_3', 'SW72_4', 'SW72_5', 'SW72_6',
       'KDCAB', 'TGLST', 'COUNTERPARTY BANK', 'BANK NAME', 'ORG_PRC_DT',
       'NOREF_BII', 'KDWIL', 'SW32A_DT', 'AUTO_DB_FG', 'STP_FG', 'EFT_AMT',
       'RECON_DT', 'K

In [133]:
tt = tt[['Periode','GCIF', 'Remark', 'SW32AMT in IDR', 'SW50_2', 'SW52A', 'SW57_1', 'SW59_2']]

In [134]:
tt.columns = ['BASE_DATE', 'GCIF', 'Remark', 'AMT', 'SENDER_NM', 'SENDER_BKCD', 'RECEIVER_BKCD', 'RECEIVER_NM']

In [135]:
tt

,BASE_DATE,GCIF,Remark,AMT,SENDER_NM,SENDER_BKCD,RECEIVER_BKCD,RECEIVER_NM
0,20221222,G205927650,Outgoing,1.120860e+07,FLORENCIA IRENA THIAN,IBBKIDJAXXX,BINAADADXXX,INTHINKING SL
1,20221208,G308983089,Outgoing,2.335125e+08,"CONIXX DISTRIBUSI INDONESIA, PT",IBBKIDJAXXX,ANZBAU3M,Frank Gerhard Stach
2,20221214,G009326067,Outgoing,2.783722e+09,"AGRO TIGA BERKAT, PT",IBBKIDJAXXX,ANZBAU3M,SWIFT AND COMPANY TRADE GROUP
3,20221214,G100672914,Outgoing,1.072005e+10,SUPERKRANE MITRA UTAMA PT,IBBKIDJAXXX,ANZBAU3M,TRT(AUST)PTY LTD
4,20221230,G309661783,Outgoing,1.277503e+08,"REXLINE ENGINEERING INDONESIA, PT",IBBKIDJAXXX,ANZBAU3M,sns engineering company Pty Ltd.
...,...,...,...,...,...,...,...,...
13089,20221230,G800405318,Incoming,3.121284e+07,ANDY SANTOSO,UBSWCHZH80A,NaN,LIE YUNITA
13090,20221230,G505093435,Incoming,9.962678e+07,ROOM DEAL PTE LTD,NaN,IBBKIDJAXXX,PT Mitra Global Holiday
13091,20221230,G103039541,Incoming,1.912014e+07,MS HENG YU XIU,NaN,IBBKIDJAXXX,A TONG
13092,20221230,G900720327,Incoming,4.667136e+08,SINMILL SINGAPORE (PTE) LTD,3582026967001,NaN,PT INDO BALAMBIT


In [136]:
ttNR = pd.merge(tt, nrCust, left_on='GCIF', right_on='GCIF_NO', how='left')

In [137]:
ttNR = ttNR[ttNR['CUST_NAME'].notnull()]

In [138]:
ott = ttNR[ttNR['Remark']=='Outgoing']

In [139]:
ott

,BASE_DATE,GCIF,Remark,AMT,SENDER_NM,SENDER_BKCD,RECEIVER_BKCD,RECEIVER_NM,GCIF_NO,CUST_NAME,LoB,REGION
1,20221208,G308983089,Outgoing,2.335125e+08,"CONIXX DISTRIBUSI INDONESIA, PT",IBBKIDJAXXX,ANZBAU3M,Frank Gerhard Stach,G308983089,"CONIXX DISTRIBUSI INDONESIA, PT",RSME,"JATIM, BALI & NUSRA"
2,20221214,G009326067,Outgoing,2.783722e+09,"AGRO TIGA BERKAT, PT",IBBKIDJAXXX,ANZBAU3M,SWIFT AND COMPANY TRADE GROUP,G009326067,"AGRO TIGA BERKAT, PT",RSME,JAKARTA 2
3,20221214,G100672914,Outgoing,1.072005e+10,SUPERKRANE MITRA UTAMA PT,IBBKIDJAXXX,ANZBAU3M,TRT(AUST)PTY LTD,G100672914,SUPERKRANE MITRA UTAMA PT,BB,JAKARTA 2
4,20221230,G309661783,Outgoing,1.277503e+08,"REXLINE ENGINEERING INDONESIA, PT",IBBKIDJAXXX,ANZBAU3M,sns engineering company Pty Ltd.,G309661783,"REXLINE ENGINEERING INDONESIA, PT",RSME,"JATIM, BALI & NUSRA"
10,20221209,G409004611,Outgoing,5.321172e+08,PT ASIA TIMUR EKSPOR IMPOR,IBBKIDJAXXX,ANZBAU3MXXX,KAURNA WINES PTY LTD,G409004611,ASIA TIMUR EKSPOR IMPOR,RSME,JAKARTA 1
...,...,...,...,...,...,...,...,...,...,...,...,...
7065,20221228,G405281890,Outgoing,8.271978e+08,"KCF INDONESIA, PT",IBBKIDJAXXX,THE JUROKU BANK LTD,KIKUCHI COLD HEADER CO LTD,G405281890,"KCF INDONESIA, PT",RSME,JAKARTA 3
7066,20221202,G406301242,Outgoing,7.783750e+08,JAPAN ASIA CONSULTANTS PT,IBBKIDJAXXX,NaN,PT. JAPAN ASIA CONSULTANTS,G406301242,JAPAN ASIA CONSULTANTS,RSME,JAKARTA 1
7067,20221205,G009003286,Outgoing,9.949032e+07,"INDO SUTECH SEJAHTERA, PT",IBBKIDJAXXX,NaN,ETIA,G009003286,"INDO SUTECH SEJAHTERA, PT",BB,JAWA TENGAH
7068,20221207,G808115612,Outgoing,6.275020e+06,YAYASAN RAJA AMPAT RESEARCH AND CON,IBBKIDJAXXX,NaN,EUROPE CARGO BV,G808115612,YAYASAN RAJA AMPAT RESEARCH AND CONSERVA,RSME,SULAWESI & IBT


In [140]:
ott = ott[['BASE_DATE', 'GCIF', 'CUST_NAME', 'LoB', 'RECEIVER_NM', 'RECEIVER_BKCD', 'REGION', 'AMT']]

In [141]:
ott['TYPE'] = 'OUTGOING'
ott['CHANNEL'] = 'REMITTANCE'

<ipython-input-141-3e95ae8dff56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ott['TYPE'] = 'OUTGOING'
<ipython-input-141-3e95ae8dff56>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ott['CHANNEL'] = 'REMITTANCE'


In [142]:
ott.columns = ['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME', 
'SENDER/RECIPIENT BIC', 'REGION', 'AMT', 'TYPE', 'CHANNEL']

In [ ]:
ott

In [144]:
itt = ttNR[ttNR['Remark']=='Incoming']

In [ ]:
itt

In [146]:
itt = itt[['BASE_DATE', 'GCIF', 'CUST_NAME', 'LoB', 'SENDER_NM', 'SENDER_BKCD', 'REGION', 'AMT']]

In [147]:
itt['TYPE'] = 'INCOMING'
itt['CHANNEL'] = 'REMITTANCE'

<ipython-input-147-40556f91e6c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  itt['TYPE'] = 'INCOMING'
<ipython-input-147-40556f91e6c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  itt['CHANNEL'] = 'REMITTANCE'


In [148]:
itt.columns = ['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME', 
'SENDER/RECIPIENT BIC', 'REGION', 'AMT', 'TYPE', 'CHANNEL']

In [ ]:
itt

# M2E

In [150]:
m2e = pd.read_excel(r'C:\Users\MR38804\Documents\EDW Transac\M2E\compile Transaction M2E_as per Dec 22_old.xlsx', sheet_name='TXN Detail Dec 22', usecols='A,P,Y,AA,AC,AH')

In [ ]:
m2e

In [152]:
m2e['Payment Date'] = m2e['Payment Date'].astype('str')

In [ ]:
m2e

In [154]:
m2e['Payment Date'] = m2e['Payment Date'].str.replace('-','')

In [155]:
m2e = m2e[m2e['Payment/Direct Debit Mode'].isin(['BI-FAST', 'Outward ACH (Online)'])]

In [ ]:
m2e

In [157]:
m2eNR = pd.merge(m2e, nrCust, left_on='gcif', right_on='GCIF_NO', how='left')

In [ ]:
m2eNR

In [159]:
m2eNR = m2eNR[m2eNR['CUST_NAME'].notnull()]

In [160]:
m2eNR['Beneficiary/ \nApplicant Bank'] = m2eNR['Beneficiary/ \nApplicant Bank'].str.upper()

<ipython-input-160-0859913b20ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2eNR['Beneficiary/ \nApplicant Bank'] = m2eNR['Beneficiary/ \nApplicant Bank'].str.upper()


In [161]:
m2eNR['Beneficiary/Applicant Name'] = m2eNR['Beneficiary/Applicant Name'].str.upper()

<ipython-input-161-1eb57a12a86f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2eNR['Beneficiary/Applicant Name'] = m2eNR['Beneficiary/Applicant Name'].str.upper()


In [162]:
m2eNR['TYPE'] = 'OUTGOING'
m2eNR1 = m2eNR[m2eNR['Payment/Direct Debit Mode']=='BI-FAST']
m2eNR1['CHANNEL'] = 'M2E BI-FAST'
m2eNR2 = m2eNR[m2eNR['Payment/Direct Debit Mode']=='Outward ACH (Online)']
m2eNR2['CHANNEL'] = 'M2E ONLINE'
m2eFinal = m2eNR1.append(m2eNR2)

<ipython-input-162-5ea01d6ba952>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2eNR['TYPE'] = 'OUTGOING'
<ipython-input-162-5ea01d6ba952>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2eNR1['CHANNEL'] = 'M2E BI-FAST'
<ipython-input-162-5ea01d6ba952>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [ ]:
m2eFinal

In [164]:
m2eFinal.columns

Index(['Payment Date', 'gcif', 'amt in idr', 'Beneficiary/ \nApplicant Bank',
       'Beneficiary/Applicant Name', 'Payment/Direct Debit Mode', 'GCIF_NO',
       'CUST_NAME', 'LoB', 'REGION', 'TYPE', 'CHANNEL'],
      dtype='object')

In [165]:
m2eFinal = m2eFinal[['Payment Date', 'GCIF_NO', 'CUST_NAME', 'LoB', 'Beneficiary/Applicant Name', 'Beneficiary/ \nApplicant Bank',
'REGION', 'amt in idr', 'TYPE', 'CHANNEL']]

In [166]:
m2eFinal.columns = ['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME', 
'SENDER/RECIPIENT BANK', 'REGION', 'AMT', 'TYPE', 'CHANNEL']

In [ ]:
m2eFinal

# M2U

In [168]:
m2u = pd.read_csv(r"C:\Users\MR38804\Documents\EDW Transac\M2U\PCE_M2U_2212.csv", sep=";", low_memory=False)

In [169]:
# Remove NA GCIF
m2u_clean = m2u[~m2u['GCIF_NO'].isnull()]

In [170]:
m2u_nr = m2u_clean[m2u_clean['SEGMENT'].isin(['CFS-NONRB-RSME', 'CFS-NONRB-SME+', 'CFS-NONRB-BB'])]

In [171]:
m2u_nr = pd.merge(m2u_nr, nrCust, left_on='GCIF_NO', right_on='GCIF_NO', how='left')

In [172]:
m2u_nr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11272 entries, 0 to 11271
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BASE_DT              11272 non-null  int64  
 1   TX_DT                11272 non-null  int64  
 2   TX_TM                11272 non-null  int64  
 3   SEQ_NO               11272 non-null  int64  
 4   TERM_NO              11272 non-null  int64  
 5   TERM_TP              11272 non-null  object 
 6   MER_ID               11272 non-null  int64  
 7   TERM_LOC             11272 non-null  object 
 8   CARD_ACCT_NO         11155 non-null  float64
 9   TX_BR_NO             11155 non-null  float64
 10  CARD_APPL            11272 non-null  int64  
 11  APPL_CD              11272 non-null  int64  
 12  FROM_ACCT_NO         11272 non-null  int64  
 13  TX_AMT               11272 non-null  float64
 14  RULT_CD              11272 non-null  int64  
 15  TERM_RCPT            11272 non-null 

In [173]:
m2u_nr[m2u_nr.duplicated()]

,BASE_DT,TX_DT,TX_TM,SEQ_NO,TERM_NO,TERM_TP,MER_ID,TERM_LOC,CARD_ACCT_NO,TX_BR_NO,...,SWITCH_IN_CITY_CTRY,SWITCH_IN_CTRY_CD,SWITCH_IN_MCC,ACQ_POS_ENTRY_MODE,APPRV_CD,GCIF_NO,SEGMENT,CUST_NAME,LoB,REGION


In [174]:
m2u_nr['BENFRY_NM'] = m2u_nr['PYMT_DATA'].str.extract('\s{2,}(\D*\S)\s+')

In [175]:
m2u_nr['BENFRY_BKCD'] = m2u_nr['BENFRY_BKCD'].astype('str').str.zfill(3)

In [176]:
m2uFinal = pd.merge(m2u_nr, BKCD, left_on='BENFRY_BKCD', right_on='BANK CODE', how='left')

In [177]:
# data with 0 as bank code
m2uFinal['BANK NAME'].isna().sum()

1833

In [178]:
m2uFinal.columns

Index(['BASE_DT', 'TX_DT', 'TX_TM', 'SEQ_NO', 'TERM_NO', 'TERM_TP', 'MER_ID',
       'TERM_LOC', 'CARD_ACCT_NO', 'TX_BR_NO', 'CARD_APPL', 'APPL_CD',
       'FROM_ACCT_NO', 'TX_AMT', 'RULT_CD', 'TERM_RCPT', 'SWITCH_ID',
       'TX_CD_ID', 'TO_ACCT_NO', 'BENFRY_BKCD', 'TO_APPL_CD', 'PYMT_CD',
       'PYMT_INSTR', 'PYMT_DATA', 'ORG_PRC_DT', 'CPY_DT', 'CPY_TM',
       'LST_MDFCPY_DT', 'LST_MDFCPY_TM', 'CURR_CD', 'RVSAL_TX_CD',
       'RVRSAL_RCPT_NO', 'SWITCH_IN_CITY_CTRY', 'SWITCH_IN_CTRY_CD',
       'SWITCH_IN_MCC', 'ACQ_POS_ENTRY_MODE', 'APPRV_CD', 'GCIF_NO', 'SEGMENT',
       'CUST_NAME', 'LoB', 'REGION', 'BENFRY_NM', 'BANK CODE', 'BANK NAME'],
      dtype='object')

In [179]:
m2uFinal = m2uFinal[['BASE_DT', 'GCIF_NO', 'CUST_NAME', 'LoB', 'BENFRY_NM', 'BANK NAME', 'REGION', 'TX_AMT']]

In [180]:
m2uFinal['TYPE'] = 'OUTGOING'
m2uFinal['CHANNEL'] = 'M2U'

In [181]:
m2uFinal.columns = ['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME', 
'SENDER/RECIPIENT BANK', 'REGION', 'AMT', 'TYPE', 'CHANNEL']

In [ ]:
m2uFinal

In [183]:
m2uFinal = m2uFinal[m2uFinal['SENDER/RECIPIENT BANK'].notna()]

In [ ]:
m2uFinal

# COMBINE ALL

In [185]:
trxAll = pd.concat([ortgsFinal, irtgsFinal, osknFinal, isknFinal,ott, itt, m2eFinal, m2uFinal], ignore_index=True)

In [ ]:
trxAll

In [187]:
trxAll.columns

Index(['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME',
       'SENDER/RECIPIENT BANK', 'REGION', 'AMT', 'TYPE', 'CHANNEL',
       'SENDER/RECIPIENT BIC'],
      dtype='object')

In [188]:
trxAll = trxAll[['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME',
       'SENDER/RECIPIENT BANK', 'REGION', 'AMT', 'TYPE', 'CHANNEL']]

In [189]:
trxAll['SENDER/RECIPIENT BANK'] = 'EXTERNAL'

<ipython-input-189-96d32ed06716>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trxAll['SENDER/RECIPIENT BANK'] = 'EXTERNAL'


In [190]:
trxAll[['CHANNEL', 'TYPE']].value_counts()

CHANNEL      TYPE    
SKN          INCOMING    218714
             OUTGOING     91662
M2E BI-FAST  OUTGOING     14313
M2U          OUTGOING      9439
RTGS         INCOMING      8808
M2E ONLINE   OUTGOING      6494
RTGS         OUTGOING      6301
REMITTANCE   OUTGOING      5615
             INCOMING      3229
dtype: int64

In [191]:
trxAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364575 entries, 0 to 364574
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   BASE DATE              364575 non-null  object 
 1   GCIF                   364575 non-null  object 
 2   CUST NAME              364575 non-null  object 
 3   LOB                    364575 non-null  object 
 4   SENDER/RECIPIENT NAME  364457 non-null  object 
 5   SENDER/RECIPIENT BANK  364575 non-null  object 
 6   REGION                 328931 non-null  object 
 7   AMT                    364575 non-null  float64
 8   TYPE                   364575 non-null  object 
 9   CHANNEL                364575 non-null  object 
dtypes: float64(1), object(9)
memory usage: 27.8+ MB


In [192]:
trxAll[['REGION']] = trxAll[['REGION']].fillna('NO DATA')

c:\Users\MR38804\Anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [193]:
trxAll.columns

Index(['BASE DATE', 'GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME',
       'SENDER/RECIPIENT BANK', 'REGION', 'AMT', 'TYPE', 'CHANNEL'],
      dtype='object')

In [194]:
trxGPD = trxAll.groupby(['GCIF', 'CUST NAME', 'LOB', 'SENDER/RECIPIENT NAME',
       'SENDER/RECIPIENT BANK', 'TYPE', 'CHANNEL', 'REGION'])['AMT'].agg(AMOUNT_TX_IDR = 'sum',TRX='count')

In [195]:
trxGPD.reset_index(inplace=True)

In [ ]:
trxGPD

# Name Cleansing

In [198]:
#simVal = []
#for row in trxGPD.index:
#    cust = nlp(u'{}'.format(trxGPD.loc[row, 'CUST NAME']))
#    ext = nlp(u'{}'.format(trxGPD.loc[row, 'SENDER/RECIPIENT NAME']))
#    result = cust.similarity(ext)
#    simVal.append(result)

In [199]:
# trxGPD['SIMILARITY'] = simVal

In [ ]:
trxGPD

In [201]:
#cleanse name values - benficiary name
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.upper()
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('.',' ')
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('-',' ')
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace(',',' ')
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('TBK',' ')
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.strip()
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('^PT',' ',regex=True)
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('PT$',' ',regex=True)
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('^CV',' ',regex=True)
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('CV$',' ',regex=True)
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.strip()
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('\sP$',' ',regex=True)
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace("''"," ")
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('//',' ')
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('\s+',' ',regex=True)
trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.strip()

<ipython-input-201-7d0363c8d712>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  trxGPD['SENDER/RECIPIENT NAME']=trxGPD['SENDER/RECIPIENT NAME'].str.replace('.',' ')


In [202]:
#cleanse name values - benficiary name
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.upper()
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('.',' ')
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('-',' ')
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace(',',' ')
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('TBK',' ')
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.strip()
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('^PT',' ',regex=True)
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('PT$',' ',regex=True)
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('^CV',' ',regex=True)
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('CV$',' ',regex=True)
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.strip()
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('\sP$',' ',regex=True)
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace("''"," ")
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('//',' ')
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('\s+',' ',regex=True)
trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.strip()

<ipython-input-202-237dfa68bac0>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  trxGPD['CUST NAME']=trxGPD['CUST NAME'].str.replace('.',' ')


In [ ]:
trxGPD.to_excel("GPD 2212 WITH TYPE.xlsx", index=False)